In [206]:
# 데이터 처리 및 분석 함수들

def process_data_for_survival(df, START_DATE, END_DATE, CUTOFF_DATE):
    """
    원본 데이터를 분석용으로 전처리
    """
    # 완료 상태 정의
    FINISHED_STATES = ['FINISH', 'AUTO_FINISH', 'DONE', 'NOCARD', 'NOPAY']

    # 1. 기간 필터링
    print(f"📆 기간 필터링: {START_DATE.strftime('%Y-%m-%d')} ~ {END_DATE.strftime('%Y-%m-%d')}")

    # 유효한 결제일이 있는 데이터만 필터링
    valid_pay_date = df['fst_pay_date'].notna()
    in_period = (
        (df['crda'] >= pd.to_datetime(START_DATE)) &
        (df['crda'] <= pd.to_datetime(END_DATE))
    )

    filtered_data = df[valid_pay_date & in_period].copy()

    # 고유한 수업 식별자 생성 (lecture_vt_No + p_rn)
    filtered_data['lesson_id'] = (filtered_data['lecture_vt_No'].astype(str) + '_' +
                                filtered_data['p_rn'].astype(str))

    print(f"✅ 기간 필터링 결과: {len(filtered_data):,}개 행 (원본의 {len(filtered_data)/len(df)*100:.1f}%)")

    # 2. 수업 완료 상태 및 done_month 보정
    def determine_status_and_correct_month(row):
        """각 행의 완료 상태 판정 및 done_month 보정"""
        churn = False
        corrected_done_month = row['done_month'] if pd.notna(row['done_month']) else 0

        # 1) 명시적 완료 상태 확인
        if row['tutoring_state'] in FINISHED_STATES:
            churn = True

        # 2) 암시적 완료 상태 확인 (ACTIVE이지만 마지막 수업일이 기준일보다 이전)
        elif (row['tutoring_state'] == 'ACTIVE' and
              pd.notna(row['lst_tutoring_datetime']) and
              row['lst_tutoring_datetime'] < CUTOFF_DATE):
            churn = True

            # done_month 보정 로직
            if (pd.notna(row['crda']) and pd.notna(row['lst_tutoring_datetime'])):
                # 실제 수업 기간 계산 (28일 = 1개월로 가정)
                actual_days = (row['lst_tutoring_datetime'] - row['crda']).days
                actual_months = actual_days / 28

                # done_month가 실제 기간보다 크면 보정
                if row['done_month'] > actual_months:
                    corrected_done_month = actual_months * 0.8  # 80%로 보정

        return pd.Series({
            'churn': churn,
            'done_month_corrected': corrected_done_month
        })

    # 상태 및 보정 적용
    status_correction = filtered_data.apply(determine_status_and_correct_month, axis=1)
    processed_data = pd.concat([filtered_data, status_correction], axis=1)

    # 결과 요약
    total_count = len(processed_data)
    finished_count = processed_data['churn'].sum()
    active_count = total_count - finished_count
    corrected_rows = (
        processed_data['done_month_corrected'] !=
        processed_data['done_month'].fillna(0)
    ).sum()

    print("📊 전처리 완료 결과:")
    print(f"   📝 총 수업 수: {total_count:,}개")
    print(f"   ❌ 중단 수업: {finished_count:,}개 ({finished_count/total_count*100:.1f}%)")
    print(f"   ✅ 활성 수업: {active_count:,}개 ({active_count/total_count*100:.1f}%)")
    print(f"   🔧 DM 보정: {corrected_rows:,}개")

    return processed_data

def perform_kaplan_meier_analysis(processed_df):
    """Kaplan-Meier 생존 분석 수행"""
    from lifelines import KaplanMeierFitter
    
    # Kaplan-Meier 피팅
    durations = processed_df["done_month_corrected"]
    events = processed_df["churn"]

    kmf = KaplanMeierFitter()
    kmf.fit(durations, event_observed=events, label="전체")

    # 생존곡선 데이터프레임 만들기
    survival_df = kmf.survival_function_.reset_index()
    survival_df.columns = ["개월", "생존확률"]

    # 36개월까지만 필터
    survival_df = survival_df[(survival_df["개월"] <= 36)&(survival_df["개월"]>=0)]

    # AUC 계산
    auc_value = np.trapz(survival_df["생존확률"], survival_df["개월"])

    return survival_df, auc_value

def create_auc_analysis_table(processed_df):
    """AUC 분석 결과 표 생성"""
    from lifelines import KaplanMeierFitter
    
    print("📊 AUC 분석 결과")

    results = []
    groups = [
        ("전체", None),
        ("1개월 구매", 1),
        ("3개월 구매", 3),
        ("6개월 구매", 6),
        ("12개월 구매", 12)
    ]

    for group_name, fst_month in groups:
        # 데이터 필터링
        if fst_month is None:
            data = processed_df
        else:
            data = processed_df[processed_df['fst_months'] == fst_month]

        if len(data) > 0:
            # 기본 통계
            sample_size = len(data)
            churn_count = data['churn'].sum()
            churn_rate = churn_count / sample_size * 100

            # Kaplan-Meier 피팅
            durations = data["done_month_corrected"]
            events = data["churn"]

            kmf = KaplanMeierFitter()
            kmf.fit(durations, event_observed=events)

            # 생존곡선 데이터 생성 (36개월까지)
            survival_df = kmf.survival_function_.reset_index()
            survival_df.columns = ["개월", "생존확률"]
            survival_df = survival_df[(survival_df["개월"] <= 36) & (survival_df["개월"] >= 0)]

            # AUC 계산
            auc_value = np.trapz(survival_df["생존확률"], survival_df["개월"])

            # 중위 생존기간 계산
            median_survival = kmf.median_survival_time_
            median_survival = median_survival if not pd.isna(median_survival) else "도달 안함"

            results.append({
                "구분": group_name,
                "샘플 수": f"{sample_size:,}개",
                "중단율": f"{churn_rate:.1f}%",
                "AUC (36개월)": f"{auc_value:.2f}개월",
                "중위 생존기간": f"{median_survival:.1f}개월" if median_survival != "도달 안함" else median_survival
            })

    # 데이터프레임으로 변환하여 표시
    results_df = pd.DataFrame(results)
    return results_df

print("✅ 함수들이 정의되었습니다.")

✅ 함수들이 정의되었습니다.


# 📊 고객 잔존기간 AUC 모델링 툴 설계 문서
## 1. 목적 (Goal)

고객 잔존기간(Residual Lifetime)을 AUC 기반으로 계산하는 모델링 툴 구축

특정 구간의 이탈률 개선이 전체 AUC에 미치는 영향을 정량적으로 추정 → 실험 투자 의사결정 기준으로 활용

실험 전(사전 시뮬레이션)과 실험 후(후행 검증) 모두에서 참고 가능한 지표 제공

## 2. 데이터 소스 및 정의

CSV 원본: lvt_done_month_수정.csv (수업 단위 데이터)

핵심 컬럼

컬럼명	정의	비고
done_month	수업 잔존기간 (개월)	AUC 시간축
lst_tutoring_datetime	마지막 수업 일시	실제 종료 기준
lst_done_at	중단 요청서 처리 일시	모델 직접 반영 X
## 3. 모델링 방식

기준 AUC 계산

Kaplan-Meier 기반, 36개월 시뮬레이션

현재 AUC ≈ 5.85 ~ 6.07개월

개선 효과 시뮬레이션

특정 구간 이탈률 개선(-x%) 반영 → 전체 곡선 업데이트

ΔAUC = 개선 후 – 기존

## 4. 분석 구간 (Convention)

결제 직후 ~ 매칭 전

매칭 직후 ~ 첫 수업 전

첫 수업 ~ 2회차

2회차 이후 ~ 1개월 이내

⚠️ 현재 모델 구간과 대시보드 컨벤션 불일치 → 수정 필요

## 5. 벤치마크 및 검증

대시보드 vs 모델 불일치

첫 수업 전 이탈률: 모델 5.25% vs 실제 ~10%

신뢰도 기준 필요

최근 2주 데이터 → 신뢰도 낮음 표시

95%/99% CI 제시

## 6. 기능 요구사항

 AUC 기본 계산기: CSV 업로드 → AUC 산출

 시뮬레이션 기능: 개선율 입력 → ΔAUC 계산

 실험 기여도 평가: 코호트 단위 영향 추정

 후행 검증 기능: 실험 전/후 비교

## 7. 개발 일정

9/26 (금): 내부 설계 점검

10/1 (수): 중간 리뷰 (with Monde)

10/17 (금): 최종 완성

## 8. 오픈 이슈

 CSV vs 대시보드 값 정합성 검증

 구간 컨벤션 표준화 필요

 ΔAUC 임계값(의사결정 기준치) 확정

## 🔧 개선 희망 포인트

구간 정의 불일치: “Critical/Early/Stable” vs 경험그룹 컨벤션

벤치마크 정합성

done_month vs done_week 변환 로직 (28일 vs 30일)

필터링 기간 차이

활성 수업 처리 방식

## 🎯 원하는 것

의사결정 기준 제공: ΔAUC ≥ X% → “Go”

성공 시 기대 AUC 확인: Kaplan-Meier 곡선 업데이트

## 🚀 작업 구상 (실행 플로우)

구간 정의 정렬

벤치마크 검증

임팩트 시뮬레이션

의사결정 기준 수립

최종 산출물:

Baseline vs To-Be 곡선 비교

구간별 생존율/이탈율 테이블

ΔAUC 증분 표

실험 추진 가능 여부 플래그

# 1.환경설정

In [185]:
import pandas as pd
import numpy as np
import utils.korean_font_setup as korean_font_setup
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [207]:
df = pd.read_csv("data/AUC기본소스2508_lvt_done_month_수정.csv")
df.head()

,crda,lecture_vt_No,student_user_No,tutoring_state,student_name,subject,grade,tteok_ham_type,reactive_datetime,p_rn,fst_pay_date,lst_done_at,done_month,lst_tutoring_datetime,reactive,fst_months
0,2020-07-26,4351,385807,ACTIVE,김혜영,영어,고3,W2_H60,NaN,1,2021-07-06 20:59:40.000,NaN,67.875,2025-08-13 23:00:00.000,not_reactive,1.0
1,2020-03-01,1984,357784,ACTIVE,천은빈,영어,고3,W1_H90,NaN,1,2021-07-17 13:33:28.000,NaN,60.000,2025-08-10 08:00:00.000,not_reactive,1.0
2,2023-02-20,33610,535001,AUTO_FINISH,강경오,수학,고3,W2_H60,2023-02-20 17:06:53.000,3,2023-02-20 17:06:53.000,2024-10-10 11:17:53.000,59.500,2024-10-10 19:35:00.000,after_reactive_datetime,6.0
3,2025-04-29,4265,385097,ACTIVE,정서윤,수학,고3,W2_H90,2025-04-29 13:07:12.000,1,2021-07-13 20:04:38.000,2024-03-11 16:28:59.000,54.375,2024-03-13 00:00:00.000,before_reactive_datetime,1.0
4,2021-06-09,14487,441759,ACTIVE,유은서,수학,고3,W2_H60,NaN,1,2021-09-01 22:50:10.000,NaN,54.125,2025-08-09 19:50:00.000,not_reactive,1.0
